In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_pr.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,774589,1724057303,4/28/20,252,PPP,CARIBBEAN TEMPORARY SERVICES LLC,PONCE DE LEON 1431 SUITE 701,SANTURCE,PR,907.0,...,999 Other,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,NaN
1,774590,3828657103,4/12/20,252,PPP,SOUTH AMERICAN RESTAURANTS CORP.,"35 Diana St, Amelia Industrial Park",GUAYNABO,PR,969.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,NaN
2,774591,4150097210,4/27/20,252,PPP,"GENESIS SECURITY SERVICES, INC.",5900 AVE. ISLA VERDE L2 PMB,CAROLINA,PR,979.0,...,999 Other,5616 Investigation and Security Services,5616.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,NaN
3,774592,5502967210,4/27/20,252,PPP,CENTRO MEDICO DEL TURABO INC.,100 AVE. LUIS MUNOZ MARIN,CAGUAS,PR,725.0,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,NaN
4,774593,7953047007,4/8/20,252,PPP,CARIBBEAN RESTAURANTS LLC,5 Carr Km 6 Hm 2 Barrio Amelia,GUAYNABO,PR,968.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,777536,6288858603,3/23/21,252,PPS,GLADIUS LLC,300 Ave Rincon de Gautier Las Vistas Shopping ...,San Juan,PR,926.0,...,999 Other,5414 Specialized Design Services,5414.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,NaN
2948,777537,8218768310,1/29/21,252,PPS,LABORATORIO CLINICO VAN SCOY,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,Bayamon,PR,957.0,...,999 Other,6215 Medical and Diagnostic Laboratories,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,NaN
2949,777538,9097578505,3/12/21,252,PPP,PERSONNEL RECRUITING SERVICE CORP,1244 Ave Munoz Rivera Reparto Universitario,Ponce,PR,717.0,...,999 Other,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",500 or more,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,NaN
2950,777539,9110348403,2/16/21,252,PPS,PANUCHO INC.,1762 Calle Loiza Esq. Diez de Andino,San Juan,PR,914.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
84,6466228606,"Carr. 5506 KM 1.8, Mercedita, PR",NaN,NaN
2862,1670129002,"Carr. #1 KM 122.4, MERCEDITA, PR",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,1724057303,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466
1,3828657103,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672
2,4150097210,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696
3,5502967210,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109
4,7953047007,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/PR_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [27]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('72')] #PR
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/2950 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

2950


Matching FIPS to DataFrame

In [10]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1724057303,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,[721270096012001]
1,3828657103,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,[720610402001013]
2,4150097210,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,[720310501131002]
3,5502967210,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,[720252018001004]
4,7953047007,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,[720610402001013]
...,...,...,...,...,...
2947,6288858603,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,[721270100022014]
2948,8218768310,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,[720210322002004]
2949,9097578505,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,[721130715003007]
2950,9110348403,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,[721270015002000]


Remove brackets

In [12]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1724057303,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,721270096012001
1,3828657103,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,720610402001013
2,4150097210,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,720310501131002
3,5502967210,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,720252018001004
4,7953047007,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,720610402001013
...,...,...,...,...,...
2947,6288858603,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,721270100022014
2948,8218768310,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,720210322002004
2949,9097578505,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,721130715003007
2950,9110348403,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,721270015002000


In [13]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
111,2954357201,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,NaN
351,2942887210,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,NaN
672,9390667707,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,NaN
1052,6116877007,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,NaN
1363,3391277310,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,NaN
1654,9603427100,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,NaN
1665,2372087203,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,NaN
1777,9332887705,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,NaN
1799,2760167303,"167 Paris Street, Hato Rey, PR",48.856614,2.352222,NaN
1829,3178908600,"CALLE R LOTE 4 EL COMANDANTE IND PARK, CAROLIN...",20.593684,78.962880,NaN


Parse for Merge

In [14]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,1724057303,721270096012001
1,3828657103,720610402001013
2,4150097210,720310501131002
3,5502967210,720252018001004
4,7953047007,720610402001013
...,...,...
2947,6288858603,721270100022014
2948,8218768310,720210322002004
2949,9097578505,721130715003007
2950,9110348403,721270015002000


Read in base data

In [15]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_pr.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/PR_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,774589,774589,774589,1724057303,4/28/20,252,PPP,CARIBBEAN TEMPORARY SERVICES LLC,PONCE DE LEON 1431 SUITE 701,SANTURCE,...,999 Other,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,NaN
1,774590,774590,774590,3828657103,4/12/20,252,PPP,SOUTH AMERICAN RESTAURANTS CORP.,"35 Diana St, Amelia Industrial Park",GUAYNABO,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,NaN
2,774591,774591,774591,4150097210,4/27/20,252,PPP,"GENESIS SECURITY SERVICES, INC.",5900 AVE. ISLA VERDE L2 PMB,CAROLINA,...,999 Other,5616 Investigation and Security Services,5616.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,NaN
3,774592,774592,774592,5502967210,4/27/20,252,PPP,CENTRO MEDICO DEL TURABO INC.,100 AVE. LUIS MUNOZ MARIN,CAGUAS,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,NaN
4,774593,774593,774593,7953047007,4/8/20,252,PPP,CARIBBEAN RESTAURANTS LLC,5 Carr Km 6 Hm 2 Barrio Amelia,GUAYNABO,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,NaN


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,774589,774589,774589,1724057303,4/28/20,252,PPP,CARIBBEAN TEMPORARY SERVICES LLC,PONCE DE LEON 1431 SUITE 701,SANTURCE,...,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,NaN,721270096012001
1,774590,774590,774590,3828657103,4/12/20,252,PPP,SOUTH AMERICAN RESTAURANTS CORP.,"35 Diana St, Amelia Industrial Park",GUAYNABO,...,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,NaN,720610402001013
2,774591,774591,774591,4150097210,4/27/20,252,PPP,"GENESIS SECURITY SERVICES, INC.",5900 AVE. ISLA VERDE L2 PMB,CAROLINA,...,5616 Investigation and Security Services,5616.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,NaN,720310501131002
3,774592,774592,774592,5502967210,4/27/20,252,PPP,CENTRO MEDICO DEL TURABO INC.,100 AVE. LUIS MUNOZ MARIN,CAGUAS,...,6221 General Medical and Surgical Hospitals,6221.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,NaN,720252018001004
4,774593,774593,774593,7953047007,4/8/20,252,PPP,CARIBBEAN RESTAURANTS LLC,5 Carr Km 6 Hm 2 Barrio Amelia,GUAYNABO,...,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,NaN,720610402001013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,777536,777536,777536,6288858603,3/23/21,252,PPS,GLADIUS LLC,300 Ave Rincon de Gautier Las Vistas Shopping ...,San Juan,...,5414 Specialized Design Services,5414.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,NaN,721270100022014
2948,777537,777537,777537,8218768310,1/29/21,252,PPS,LABORATORIO CLINICO VAN SCOY,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,Bayamon,...,6215 Medical and Diagnostic Laboratories,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,NaN,720210322002004
2949,777538,777538,777538,9097578505,3/12/21,252,PPP,PERSONNEL RECRUITING SERVICE CORP,1244 Ave Munoz Rivera Reparto Universitario,Ponce,...,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",500 or more,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,NaN,721130715003007
2950,777539,777539,777539,9110348403,2/16/21,252,PPS,PANUCHO INC.,1762 Calle Loiza Esq. Diez de Andino,San Juan,...,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,NaN,721270015002000


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,774589,774589,774589,1724057303,4/28/20,252,PPP,CARIBBEAN TEMPORARY SERVICES LLC,PONCE DE LEON 1431 SUITE 701,SANTURCE,...,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,NaN,721270096012001,721270096012001
1,774590,774590,774590,3828657103,4/12/20,252,PPP,SOUTH AMERICAN RESTAURANTS CORP.,"35 Diana St, Amelia Industrial Park",GUAYNABO,...,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,NaN,720610402001013,720610402001013
2,774591,774591,774591,4150097210,4/27/20,252,PPP,"GENESIS SECURITY SERVICES, INC.",5900 AVE. ISLA VERDE L2 PMB,CAROLINA,...,5616.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,NaN,720310501131002,720310501131002
3,774592,774592,774592,5502967210,4/27/20,252,PPP,CENTRO MEDICO DEL TURABO INC.,100 AVE. LUIS MUNOZ MARIN,CAGUAS,...,6221.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,NaN,720252018001004,720252018001004
4,774593,774593,774593,7953047007,4/8/20,252,PPP,CARIBBEAN RESTAURANTS LLC,5 Carr Km 6 Hm 2 Barrio Amelia,GUAYNABO,...,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,NaN,720610402001013,720610402001013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,777536,777536,777536,6288858603,3/23/21,252,PPS,GLADIUS LLC,300 Ave Rincon de Gautier Las Vistas Shopping ...,San Juan,...,5414.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,NaN,721270100022014,721270100022014
2948,777537,777537,777537,8218768310,1/29/21,252,PPS,LABORATORIO CLINICO VAN SCOY,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,Bayamon,...,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,NaN,720210322002004,720210322002004
2949,777538,777538,777538,9097578505,3/12/21,252,PPP,PERSONNEL RECRUITING SERVICE CORP,1244 Ave Munoz Rivera Reparto Universitario,Ponce,...,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",500 or more,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,NaN,721130715003007,721130715003007
2950,777539,777539,777539,9110348403,2/16/21,252,PPS,PANUCHO INC.,1762 Calle Loiza Esq. Diez de Andino,San Juan,...,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,NaN,721270015002000,721270015002000


Drop Extra FIPS Columns

In [18]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,774589,774589,774589,1724057303,4/28/20,252,PPP,CARIBBEAN TEMPORARY SERVICES LLC,PONCE DE LEON 1431 SUITE 701,SANTURCE,...,999 Other,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,721270096012001
1,774590,774590,774590,3828657103,4/12/20,252,PPP,SOUTH AMERICAN RESTAURANTS CORP.,"35 Diana St, Amelia Industrial Park",GUAYNABO,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,720610402001013
2,774591,774591,774591,4150097210,4/27/20,252,PPP,"GENESIS SECURITY SERVICES, INC.",5900 AVE. ISLA VERDE L2 PMB,CAROLINA,...,999 Other,5616 Investigation and Security Services,5616.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,720310501131002
3,774592,774592,774592,5502967210,4/27/20,252,PPP,CENTRO MEDICO DEL TURABO INC.,100 AVE. LUIS MUNOZ MARIN,CAGUAS,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,720252018001004
4,774593,774593,774593,7953047007,4/8/20,252,PPP,CARIBBEAN RESTAURANTS LLC,5 Carr Km 6 Hm 2 Barrio Amelia,GUAYNABO,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,720610402001013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,777536,777536,777536,6288858603,3/23/21,252,PPS,GLADIUS LLC,300 Ave Rincon de Gautier Las Vistas Shopping ...,San Juan,...,999 Other,5414 Specialized Design Services,5414.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,721270100022014
2948,777537,777537,777537,8218768310,1/29/21,252,PPS,LABORATORIO CLINICO VAN SCOY,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,Bayamon,...,999 Other,6215 Medical and Diagnostic Laboratories,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,720210322002004
2949,777538,777538,777538,9097578505,3/12/21,252,PPP,PERSONNEL RECRUITING SERVICE CORP,1244 Ave Munoz Rivera Reparto Universitario,Ponce,...,999 Other,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",500 or more,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,721130715003007
2950,777539,777539,777539,9110348403,2/16/21,252,PPS,PANUCHO INC.,1762 Calle Loiza Esq. Diez de Andino,San Juan,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,721270015002000


In [19]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
84,774673,774673,774673,6466228606,3/23/21,252,PPS,SERRALLES HOTEL INC.,Carr. 5506 KM 1.8,Mercedita,...,999 Other,7211 Traveler Accommodation,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"Carr. 5506 KM 1.8, Mercedita, PR",NaN,NaN,NaN
111,774700,774700,774700,2954357201,4/16/20,252,PPP,"PRIME JANITORIAL SERVICES, CORP",URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M2,PONCE,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,500 or more,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,NaN
351,774940,774940,774940,2942887210,4/16/20,252,PPP,"PRIME JANITORIAL METRO & HEALTH, INC",URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2,PONCE,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,PUERTO RICO AND USVI DISTRICT OFFICE,"b $350,000-1 million",250 to 499,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,NaN
672,775261,775261,775261,9390667707,5/1/20,252,PPP,GENCO GENERAL CONTRACTORS OF PUERTO RICO CORP.,1402 N CORINTH ST RD SUITE 113,DAllas,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,PUERTO RICO AND USVI DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,NaN
1052,775641,775641,775641,6116877007,4/6/20,252,PPP,PARKVILLE ELDERLY HOME,Urb. Parkville # 1 Mejico St.,GUAYNABO,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,PUERTO RICO AND USVI DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,NaN
1363,775952,775952,775952,3391277310,4/29/20,252,PPP,SOUTH WESTERN EDUCATION SOCIETY (SESO) SOCIEDA...,CARRETERA 108 INTERIOR KM 3.2 BO CUBA,MAYAGUEZ,...,999 Other,6114 Business Schools and Computer and Managem...,6114.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,NaN
1654,776243,776243,776243,9603427100,4/15/20,252,PPP,VENTURE DISTRIBUTORS CORP.,Frances 39 Amelia Industrial Park,GUAYNABO,...,999 Other,4231 Motor Vehicle and Motor Vehicle Parts and...,4231.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,NaN
1665,776254,776254,776254,2372087203,4/16/20,252,PPP,QUINONES SELF SERVICE LLC,"4ta sec levittown , Centro Comercial Lago Vista",TOA BAJA,...,999 Other,4451 Grocery Stores,4451.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,NaN
1777,776366,776366,776366,9332887705,5/1/20,252,PPP,SIMPLEMENT INC,"1 Northfield Plaza, Suite 300",Northfield,...,999 Other,5112 Software Publishers,5112.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,NaN
1799,776388,776388,776388,2760167303,4/29/20,252,PPP,COLEGIO CONGREGACIN MITA INCORPORADO,167 Paris Street,Hato Rey,...,999 Other,6111 Elementary and Secondary Schools,6111.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"167 Paris Street, Hato Rey, PR",48.856614,2.352222,NaN


Convert to File

In [20]:
fips_PR = fips_merge_drop

In [21]:
fips_PR.to_csv("../../data/state_data/geo/geo_fips/PR_fips_scraped.csv") 

Review & Compare

In [22]:
geo_pr_fips = pd.read_csv("../../data/state_data/geo/geo_fips/PR_fips_scraped.csv") 
geo_pr_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,774589,774589,774589,1724057303,4/28/20,252,PPP,CARIBBEAN TEMPORARY SERVICES LLC,PONCE DE LEON 1431 SUITE 701,...,999 Other,5613 Employment Services,5613.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,7.212701e+14
1,1,774590,774590,774590,3828657103,4/12/20,252,PPP,SOUTH AMERICAN RESTAURANTS CORP.,"35 Diana St, Amelia Industrial Park",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,7.206104e+14
2,2,774591,774591,774591,4150097210,4/27/20,252,PPP,"GENESIS SECURITY SERVICES, INC.",5900 AVE. ISLA VERDE L2 PMB,...,999 Other,5616 Investigation and Security Services,5616.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,7.203105e+14
3,3,774592,774592,774592,5502967210,4/27/20,252,PPP,CENTRO MEDICO DEL TURABO INC.,100 AVE. LUIS MUNOZ MARIN,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,7.202520e+14
4,4,774593,774593,774593,7953047007,4/8/20,252,PPP,CARIBBEAN RESTAURANTS LLC,5 Carr Km 6 Hm 2 Barrio Amelia,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,500 or more,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,7.206104e+14


In [23]:
##### fill in missing Lat, Long, FIPS
geo_pr_fips.loc[geo_pr_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
84,84,774673,774673,774673,6466228606,3/23/21,252,PPS,SERRALLES HOTEL INC.,Carr. 5506 KM 1.8,...,999 Other,7211 Traveler Accommodation,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"Carr. 5506 KM 1.8, Mercedita, PR",NaN,NaN,NaN
111,111,774700,774700,774700,2954357201,4/16/20,252,PPP,"PRIME JANITORIAL SERVICES, CORP",URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M2,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,500 or more,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,NaN
351,351,774940,774940,774940,2942887210,4/16/20,252,PPP,"PRIME JANITORIAL METRO & HEALTH, INC",URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,PUERTO RICO AND USVI DISTRICT OFFICE,"b $350,000-1 million",250 to 499,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,NaN
672,672,775261,775261,775261,9390667707,5/1/20,252,PPP,GENCO GENERAL CONTRACTORS OF PUERTO RICO CORP.,1402 N CORINTH ST RD SUITE 113,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,PUERTO RICO AND USVI DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,NaN
1052,1052,775641,775641,775641,6116877007,4/6/20,252,PPP,PARKVILLE ELDERLY HOME,Urb. Parkville # 1 Mejico St.,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,PUERTO RICO AND USVI DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,NaN
1363,1363,775952,775952,775952,3391277310,4/29/20,252,PPP,SOUTH WESTERN EDUCATION SOCIETY (SESO) SOCIEDA...,CARRETERA 108 INTERIOR KM 3.2 BO CUBA,...,999 Other,6114 Business Schools and Computer and Managem...,6114.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,NaN
1654,1654,776243,776243,776243,9603427100,4/15/20,252,PPP,VENTURE DISTRIBUTORS CORP.,Frances 39 Amelia Industrial Park,...,999 Other,4231 Motor Vehicle and Motor Vehicle Parts and...,4231.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,NaN
1665,1665,776254,776254,776254,2372087203,4/16/20,252,PPP,QUINONES SELF SERVICE LLC,"4ta sec levittown , Centro Comercial Lago Vista",...,999 Other,4451 Grocery Stores,4451.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,NaN
1777,1777,776366,776366,776366,9332887705,5/1/20,252,PPP,SIMPLEMENT INC,"1 Northfield Plaza, Suite 300",...,999 Other,5112 Software Publishers,5112.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,NaN
1799,1799,776388,776388,776388,2760167303,4/29/20,252,PPP,COLEGIO CONGREGACIN MITA INCORPORADO,167 Paris Street,...,999 Other,6111 Elementary and Secondary Schools,6111.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"167 Paris Street, Hato Rey, PR",48.856614,2.352222,NaN


In [39]:
geo_pr_fips["BusinessType"] = geo_pr_fips["BusinessType"].fillna('Not Available')
geo_pr_fips["BusinessType"].unique()
geo_pr_fips.to_csv("../../data/state_data/geo/geo_fips/PR_fips_scraped.csv") 
geo_pr_fips =pd.read_csv("../../data/state_data/geo/geo_fips/PR_fips_scraped.csv") 
geo_pr_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Subchapter S Corporation',
       'Corporation', 'Partnership', 'Limited Liability Partnership',
       '501(c)3 – Non Profit', 'Employee Stock Ownership Plan(ESOP)',
       'Professional Association', 'Sole Proprietorship',
       'Non-Profit Organization', 'Not Available', 'Joint Venture',
       'Cooperative', '501(c)6 – Non Profit Membership',
       'Self-Employed Individuals', 'Trust', 'Independent Contractors',
       'Non-Profit Childcare Center', 'Single Member LLC',
       'Tenant in Common', 'Qualified Joint-Venture (spouses)'],
      dtype=object)